## use website text as one view and linkedin description as another view

In [1]:
import pandas as pd, numpy as np

In [2]:
df = pd.read_excel('USA_SAAS_CB_10Dec16.xlsx')
df.shape

(2593, 20)

In [3]:
df = df.dropna(subset=['category_list'])
df.index = range(len(df))
df.shape

(2572, 20)

In [4]:
df['enterprise_software_present'] = df['category_list'].apply(lambda x:1 if x and 'enterprise software' in x else 0)
df['enterprise_software_present'].value_counts()

0    1933
1     639
Name: enterprise_software_present, dtype: int64

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import nltk,re
from scipy.sparse import hstack

In [6]:
df1 = pd.read_csv('USA_SAAS_CB_10Dec16_website_text.csv')
df1['website_text'] = df1['website_text'].fillna('').apply(lambda x: re.sub('\n',' ',x.lower()))
df1.shape

(2056, 2)

In [7]:
import tldextract
df1['domain_cleaned'] = df1['website'].apply(lambda x: tldextract.extract(x.lower()).domain)
df1.tail()

No handlers could be found for logger "tldextract"


,website,website_text,domain_cleaned
2051,http://www.cisco.com/web/about/ac49/ac0/ac1/ac...,cisco announces acquisition of thinksmart tech...,cisco
2052,http://macheen.com,www.macheen.com,macheen
2053,http://www.rtpholdings.com,rtp holdings - rtp holdings home solution bene...,rtpholdings
2054,http://seatserve.com,seatserve - stay seated. in-seat delivery at y...,seatserve
2055,http://www.contentlaunch.com/,content marketing software | content writing s...,contentlaunch


In [8]:
df1 = df1.drop_duplicates('domain_cleaned')
df1.shape

(1906, 3)

In [9]:
df['domain_cleaned'] = df['homepage_url'].apply(lambda x: tldextract.extract(x.lower()).domain)
df = df.drop_duplicates('domain_cleaned')
df.shape

(2391, 22)

In [10]:
df_final = pd.merge(df,df1,on='domain_cleaned')
df.shape,df1.shape,df_final.shape

((2391, 22), (1906, 3), (1893, 24))

In [11]:
df_final.tail()

,li_cpy_name,country_code,state_code,region,city,category_list,category_group_list,industry,short_description,description,...,funding_total_usd,last_funding_on,homepage_url,linkedin_url,domain,email,enterprise_software_present,domain_cleaned,website,website_text
1888,NuORDER,USA,CA,Los Angeles,West Hollywood,b2b|e-commerce|fashion|internet|saas|wholesale,commerce and shopping|design|internet services,Wholesale,NuORDER is a cloud & mobile B2B eCommerce plat...,NuORDER empowers B2B eCommerce sites for over ...,...,13900000.0,2015-02-01 00:00:00,http://www.nuorder.com,http://www.linkedin.com/company/2779405,nuorder.com,info@nuorder.com,0,nuorder,http://www.nuorder.com,nuorder #1 b2b ecommerce solution brands retai...
1889,EvoNexus (CommNexus),USA,CA,San Diego,La Jolla,big data|cyber security|hardware|life science|...,biotechnology|data and analytics|hardware|info...,Wireless,EvoNexus is a non-profit technology incubator ...,"EvoNexus (formerly CommNexus, formerly the Tel...",...,NaN,NaN,http://www.evonexus.org,http://www.linkedin.com/company/commnexus,evonexus.org,NaN,0,evonexus,http://www.evonexus.org,evonexus jobs contact us resources about about...
1890,Macheen Inc,USA,TX,Austin,Austin,enterprise software|internet|saas,internet services|software,Wireless,Macheen is a SaaS-based platform that brings m...,"Macheen, Inc is a SaaS company that brings mob...",...,34394995.0,2013-12-01 00:00:00,http://macheen.com,https://www.linkedin.com/company/macheen-inc,macheen.com,info@macheen.com,1,macheen,http://macheen.com,www.macheen.com
1891,RTP Holdings,USA,PA,Philadelphia,Radnor,saas,NaN,Wireless,"Indoor location services, strategy, solutions....","Indoor GPS makes everyday activities faster, e...",...,NaN,NaN,http://www.rtpholdings.com,http://www.linkedin.com/company/rtp-holdings,rtpholdings.com,NaN,0,rtpholdings,http://www.rtpholdings.com,rtp holdings - rtp holdings home solution bene...
1892,SeatServe,USA,NY,New York City,New York,e-commerce|mobile|sports,commerce and shopping|mobile|sports,Wireless,A real-time delivery SaaS solution designed fo...,SeatServe is a digital solutions provider base...,...,NaN,NaN,http://seatserve.com,https://www.linkedin.com/company/seatserve,seatserve.com,info@seatserve.com,0,seatserve,http://seatserve.com,seatserve - stay seated. in-seat delivery at y...


In [25]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegressionCV,LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC,SVC
from sklearn_cotraining.classifiers import CoTrainingClassifier
import sklearn_cotraining.classifiers
reload(sklearn_cotraining.classifiers)

<module 'sklearn_cotraining.classifiers' from 'sklearn_cotraining/classifiers.pyc'>

In [19]:
# train test split
np.random.seed(5)
msk = np.random.rand(len(df_final)) < 0.8
train = df_final[msk]
test = df_final[~msk]
train.index = range(len(train))
test.index = range(len(test))

tfidf_vectorizer_descr = TfidfVectorizer(max_df=0.8, min_df=0.01,stop_words='english',ngram_range=(1,2))
X_descr_train = tfidf_vectorizer_descr.fit_transform(train['description'].fillna(''))
X_descr_test = tfidf_vectorizer_descr.transform(test['description'].fillna(''))

tfidf_vectorizer_website_text = TfidfVectorizer(max_df=0.8, min_df=0.01,stop_words='english',ngram_range=(1,2))
X_website_text_train = tfidf_vectorizer_website_text.fit_transform(train['website_text'].fillna(''))
X_website_text_test = tfidf_vectorizer_website_text.transform(test['website_text'].fillna(''))

print X_descr_train.shape,X_website_text_train.shape,X_descr_test.shape,X_website_text_test.shape

(1506, 1199) (1506, 7435) (387, 1199) (387, 7435)


In [20]:
train = df_final[msk]
test = df_final[~msk]
train.index = range(len(train))
test.index = range(len(test))
y_train,y_test = train['enterprise_software_present'],test['enterprise_software_present']
y_train_copy = y_train.copy()
y_train.value_counts()

0    1117
1     389
Name: enterprise_software_present, dtype: int64

In [21]:
#setting random 70% y_train as -1 (unlabelled)
np.random.seed(5)
msk1 = np.random.rand(len(y_train)) < 0.7
y_train[msk1] = -1
X_train_labelled = hstack([X_descr_train[~msk1,:],X_website_text_train[~msk1,:]])
y_train_labelled = y_train[~msk1]
X_test = hstack([X_descr_test,X_website_text_test])
y_train.value_counts()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


-1    1051
 0     335
 1     120
Name: enterprise_software_present, dtype: int64

In [28]:
print '\n\nnaive bayes'
clf = MultinomialNB(fit_prior=False)
clf.fit(X_train_labelled, y_train_labelled)
print('    training set unlabelled')
y_pred = clf.predict(hstack([X_descr_train[msk1,:],X_website_text_train[msk1,:]]))
print classification_report(y_train_copy[msk1], y_pred)
print('    testing set')
y_pred = clf.predict(X_test)
print classification_report(y_test, y_pred)

print '\n\nnaive bayes cotraining'
clf = sklearn_cotraining.classifiers.CoTrainingClassifier(MultinomialNB(fit_prior=False), 
                                                                u=len(y_train)//10,k=1000)
clf.fit(X_website_text_train, X_descr_train, y_train)
print('    training set unlabelled')
y_pred = clf.predict(X_website_text_train[msk1,:], X_descr_train[msk1,:])
print classification_report(y_train_copy[msk1], y_pred)
print('    testing set')
y_pred = clf.predict(X_website_text_test, X_descr_test)
print classification_report(y_test, y_pred)

print '\n\nLogistic'
base_lr = LogisticRegressionCV(class_weight='balanced',Cs=10)
base_lr.fit(X_train_labelled, y_train_labelled)
print('    training set unlabelled')
y_pred = base_lr.predict(hstack([X_descr_train[msk1,:],X_website_text_train[msk1,:]]))
print classification_report(y_train_copy[msk1], y_pred)
print('    testing set')
y_pred = base_lr.predict(X_test)
print classification_report(y_test, y_pred)

print '\n\nLogistic CoTraining'
lg_co_clf = sklearn_cotraining.classifiers.CoTrainingClassifier(LogisticRegression(class_weight='balanced',
                    C=base_lr.C_[0]),u=len(y_train)//10,k=1000)
lg_co_clf.fit(X_website_text_train, X_descr_train, y_train)
print('    training set unlabelled')
y_pred = lg_co_clf.predict(X_website_text_train[msk1,:], X_descr_train[msk1,:])
print classification_report(y_train_copy[msk1], y_pred)
print('    testing set')
y_pred = lg_co_clf.predict(X_website_text_test, X_descr_test)
print classification_report(y_test, y_pred)




naive bayes
    training set unlabelled
             precision    recall  f1-score   support

          0       0.74      1.00      0.85       782
          1       0.33      0.00      0.01       269

avg / total       0.64      0.74      0.64      1051

    testing set
             precision    recall  f1-score   support

          0       0.78      0.99      0.87       302
          1       0.25      0.01      0.02        85

avg / total       0.66      0.78      0.69       387



naive bayes cotraining
no of iterations took for fitting:176
    training set unlabelled
             precision    recall  f1-score   support

          0       0.75      0.81      0.78       782
          1       0.29      0.23      0.26       269

avg / total       0.64      0.66      0.65      1051

    testing set
             precision    recall  f1-score   support

          0       0.80      0.83      0.81       302
          1       0.31      0.27      0.29        85

avg / total       0.69      0

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


    training set unlabelled
             precision    recall  f1-score   support

          0       0.77      0.86      0.81       782
          1       0.37      0.25      0.30       269

avg / total       0.67      0.70      0.68      1051

    testing set
             precision    recall  f1-score   support

          0       0.80      0.86      0.83       302
          1       0.34      0.26      0.30        85

avg / total       0.70      0.73      0.71       387



Logistic CoTraining
no of iterations took for fitting:158
    training set unlabelled
             precision    recall  f1-score   support

          0       0.79      0.76      0.77       782
          1       0.36      0.39      0.38       269

avg / total       0.68      0.67      0.67      1051

    testing set
             precision    recall  f1-score   support

          0       0.81      0.75      0.78       302
          1       0.30      0.38      0.34        85

avg / total       0.70      0.67      0.68    

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [29]:
print 'classifiers not using balanced class option'
print '\n\nnaive bayes'
clf = MultinomialNB(fit_prior=True)
clf.fit(X_train_labelled, y_train_labelled)
print('    training set unlabelled')
y_pred = clf.predict(hstack([X_descr_train[msk1,:],X_website_text_train[msk1,:]]))
print classification_report(y_train_copy[msk1], y_pred)
print('    testing set')
y_pred = clf.predict(X_test)
print classification_report(y_test, y_pred)

print '\n\nnaive bayes cotraining'
clf = sklearn_cotraining.classifiers.CoTrainingClassifier(MultinomialNB(fit_prior=True), 
                                                                u=len(y_train)//10,k=1000)
clf.fit(X_website_text_train, X_descr_train, y_train)
print('    training set unlabelled')
y_pred = clf.predict(X_website_text_train[msk1,:], X_descr_train[msk1,:])
print classification_report(y_train_copy[msk1], y_pred)
print('    testing set')
y_pred = clf.predict(X_website_text_test, X_descr_test)
print classification_report(y_test, y_pred)

print '\n\nLogistic'
base_lr = LogisticRegressionCV(Cs=10)
base_lr.fit(X_train_labelled, y_train_labelled)
print('    training set unlabelled')
y_pred = base_lr.predict(hstack([X_descr_train[msk1,:],X_website_text_train[msk1,:]]))
print classification_report(y_train_copy[msk1], y_pred)
print('    testing set')
y_pred = base_lr.predict(X_test)
print classification_report(y_test, y_pred)

print '\n\nLogistic CoTraining'
lg_co_clf = sklearn_cotraining.classifiers.CoTrainingClassifier(LogisticRegression(
                    C=base_lr.C_[0]),u=len(y_train)//10,k=1000)
lg_co_clf.fit(X_website_text_train, X_descr_train, y_train)
print('    training set unlabelled')
y_pred = lg_co_clf.predict(X_website_text_train[msk1,:], X_descr_train[msk1,:])
print classification_report(y_train_copy[msk1], y_pred)
print('    testing set')
y_pred = lg_co_clf.predict(X_website_text_test, X_descr_test)
print classification_report(y_test, y_pred)


classifiers not using balanced class option


naive bayes
    training set unlabelled
             precision    recall  f1-score   support

          0       0.74      1.00      0.85       782
          1       0.00      0.00      0.00       269

avg / total       0.55      0.74      0.63      1051

    testing set
             precision    recall  f1-score   support

          0       0.78      1.00      0.88       302
          1       0.00      0.00      0.00        85

avg / total       0.61      0.78      0.68       387



naive bayes cotraining


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


no of iterations took for fitting:211
    training set unlabelled
             precision    recall  f1-score   support

          0       0.74      1.00      0.85       782
          1       0.00      0.00      0.00       269

avg / total       0.55      0.74      0.63      1051

    testing set
             precision    recall  f1-score   support

          0       0.78      1.00      0.88       302
          1       0.00      0.00      0.00        85

avg / total       0.61      0.78      0.68       387



Logistic


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


    training set unlabelled
             precision    recall  f1-score   support

          0       0.74      1.00      0.85       782
          1       0.00      0.00      0.00       269

avg / total       0.55      0.74      0.63      1051

    testing set
             precision    recall  f1-score   support

          0       0.78      1.00      0.88       302
          1       0.00      0.00      0.00        85

avg / total       0.61      0.78      0.68       387



Logistic CoTraining
no of iterations took for fitting:215
    training set unlabelled
             precision    recall  f1-score   support

          0       0.74      1.00      0.85       782
          1       0.00      0.00      0.00       269

avg / total       0.55      0.74      0.63      1051

    testing set
             precision    recall  f1-score   support

          0       0.78      1.00      0.88       302
          1       0.00      0.00      0.00        85

avg / total       0.61      0.78      0.68    

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


### conclusion
co-training does not improve overall classifier performance, but co-training gives better performance for the 
positive class. So this seems an idea worth pursuing. We need to give balanced class option though